In [1]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_DB = data_root / "건축물대장_2025년_02월.db"

건축허브에서 제공하는 건축물대장 데이터를 활용하여 분석

2025년 2월 이전 데이터는 위반건축물 등 일부 건축물이 빠져있는 문제가 있어 부득이 2025년 2월 말 데이터로 2024년 말 기준 건축물 현황을 집계

In [7]:
# Open a DuckDB connection
con = duckdb.connect(database=path_DB)

# print the list of tables in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])
    print()

# show heads of the tables
for table in tables:
    table_name = table[0]
    print(f"Head of {table_name}:")
    df = con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf()
    print(df)
    print()

Tables in the database:
기본개요

층별개요

표제부

Head of 기본개요:
  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0      100211           None        1         일반        1      총괄표제부   
1      100215           None        1         일반        1      총괄표제부   
2      100216           None        1         일반        1      총괄표제부   
3      100218           None        1         일반        1      총괄표제부   
4      100219           None        1         일반        1      총괄표제부   

                  대지_위치                  도로명_대지_위치  건물_명 시군구_코드  ...  \
0  서울특별시 종로구 관철동 12-1번지   서울특별시 종로구 종로14길 20 (관철동)  None  11110  ...   
1  서울특별시 종로구 청진동 21-1번지  서울특별시 종로구 종로5길 32-6 (청진동)  None  11110  ...   
2  서울특별시 종로구 중학동 16-2번지                       None  None  11110  ...   
3  서울특별시 종로구 중학동 24-1번지      서울특별시 종로구 율곡로 5 (중학동)  None  11110  ...   
4  서울특별시 종로구 중학동 35-1번지                       None  None  11110  ...   

  새주소_지상지하_코드 새주소_본_번 새주소_부_번 지역_코드 지구_코드 구역_코드 지역_코드_명  지구_코드_명 구역_코드_명  \
0  

In [ ]:
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the tables and count the records
for table in tables:
    table_name = table[0]
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    table_counts.append({"Table Name": table_name, "Record Count": count})

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,Table Name,Record Count
0,기본개요,27994157
1,층별개요,21048715
2,표제부,8027067


각 테이블별 레코드 수 확인

In [ ]:
# Create a dictionary to store the results
max_lengths = {}

# Iterate through the tables
for table in tables:
    table_name = table[0]
    # Fetch the table schema
    schema = con.execute(f"DESCRIBE {table_name}").fetchdf()
    # display(schema)
    # Filter text columns
    text_columns = schema[
        schema["column_type"].str.contains("VARCHAR|TEXT", case=False)
    ]["column_name"]
    # Initialize a dictionary for the current table
    max_lengths[table_name] = {}
    # Calculate max length for each text column
    for column in text_columns:
        max_length = con.execute(
            f"SELECT MAX(LENGTH({column})) FROM {table_name}"
        ).fetchone()[0]
        max_lengths[table_name][column] = max_length

# Display the results
for table_name, columns in max_lengths.items():
    print(f"Table: {table_name}")
    for column, length in columns.items():
        print(f"  Column: {column}, Max Length: {length}")
    print()

Table: 기본개요
  Column: 관리_건축물대장_PK, Max Length: 22
  Column: 관리_상위_건축물대장_PK, Max Length: 22
  Column: 대장_구분_코드, Max Length: 1
  Column: 대장_구분_코드_명, Max Length: 2
  Column: 대장_종류_코드, Max Length: 1
  Column: 대장_종류_코드_명, Max Length: 5
  Column: 대지_위치, Max Length: 33
  Column: 도로명_대지_위치, Max Length: 43
  Column: 건물_명, Max Length: 62
  Column: 시군구_코드, Max Length: 5
  Column: 법정동_코드, Max Length: 5
  Column: 대지_구분_코드, Max Length: 1
  Column: 번, Max Length: 4
  Column: 지, Max Length: 4
  Column: 특수지_명, Max Length: 100
  Column: 블록, Max Length: 11
  Column: 로트, Max Length: 10
  Column: 새주소_도로_코드, Max Length: 12
  Column: 새주소_법정동_코드, Max Length: 5
  Column: 새주소_지상지하_코드, Max Length: 1
  Column: 지역_코드, Max Length: 6
  Column: 지구_코드, Max Length: 6
  Column: 구역_코드, Max Length: 6
  Column: 지역_코드_명, Max Length: 25
  Column: 지구_코드_명, Max Length: 28
  Column: 구역_코드_명, Max Length: 23
  Column: 생성_일자, Max Length: 8

Table: 층별개요
  Column: 관리_건축물대장_PK, Max Length: 22
  Column: 대지_위치, Max Length: 33
  Column:

각 테이블별, 컬럼별 최대 문자열 길이 검사

데이터와 함께 제공된 스키마와 비교하여 데이터에 존재하는 최대 길이를 넘지 않는지 검증

(데이터 로딩이 잘못된 경우 최대 길이를 넘는 컬럼이 존재할 수 있어 확인함)

(내부DB에서 표제부와 폐말소대장이 하나의 테이블로 합쳐진 것은 개방데이터와는 무관함 확인)

In [ ]:
# Query to count the occurrences
query = """
    SELECT 
        COUNT(관리_건축물대장_PK) AS 관리_건축물대장_PK_count,
        COUNT(관리_상위_건축물대장_PK) AS 관리_상위_건축물대장_PK_count
    FROM 기본개요
"""

# Execute the query and fetch the result
result = con.execute(query).fetchdf()
print(result)

   관리_건축물대장_PK_count  관리_상위_건축물대장_PK_count
0           27994157              21493611


In [ ]:
# 1) 테이블 스키마에서 컬럼 이름을 가져오고
cols_df = con.execute("PRAGMA table_info('기본개요')").fetchdf()
cols = cols_df["name"].tolist()

# 2) 각 컬럼에 대한 COUNT 문을 생성
count_exprs = [f"COUNT({col}) AS {col}_count" for col in cols]

# 3) 최종 쿼리 조립
query = """
SELECT
    {exprs}
FROM 기본개요
""".format(exprs=",\n    ".join(count_exprs))

# 4) 실행 및 결과 확인
result = con.execute(query).fetchdf()
print(result.T)


                             0
관리_건축물대장_PK_count     27994157
관리_상위_건축물대장_PK_count  21493611
대장_구분_코드_count        27994157
대장_구분_코드_명_count      27994103
대장_종류_코드_count        27994157
대장_종류_코드_명_count      27994157
대지_위치_count           27994157
도로명_대지_위치_count       26756297
건물_명_count            18875815
시군구_코드_count          27994156
법정동_코드_count          27994152
대지_구분_코드_count        27994148
번_count               27994156
지_count               27994138
특수지_명_count             296730
블록_count                165483
로트_count                 57066
외필지_수_count           27994157
새주소_도로_코드_count       26756297
새주소_법정동_코드_count      26749968
새주소_지상지하_코드_count     27983073
새주소_본_번_count         26895824
새주소_부_번_count         15251098
지역_코드_count            4425434
지구_코드_count             863337
구역_코드_count             970081
지역_코드_명_count          4400740
지구_코드_명_count           842140
구역_코드_명_count           956922
생성_일자_count           27994157


(기본개요에서 각 컬럼별 널이 아닌 값 수를 셈)

In [ ]:
# 1) 테이블 스키마에서 컬럼 이름을 가져오고
cols_df = con.execute("PRAGMA table_info('표제부')").fetchdf()
cols = cols_df["name"].tolist()

# 2) 각 컬럼에 대한 COUNT 문을 생성
# 표제부에는 컬럼명에 %가 들어간 경우가 있어, 오류가 발생함
# 이를 피하기 위해, 컬럼명을 따옴표로 감싸줌
count_exprs = [f'COUNT("{col}") AS "{col}_count"' for col in cols]

# 3) 최종 쿼리 조립
query = """
SELECT
    {exprs}
FROM 표제부
""".format(exprs=",\n    ".join(count_exprs))

# 4) 실행 및 결과 확인
result = con.execute(query).fetchdf()

with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    # Display the result with all rows and columns
    print(result.T)

                            0
관리_건축물대장_PK_count     8027067
대장_구분_코드_count        8027067
대장_구분_코드_명_count      8027067
대장_종류_코드_count        8027067
대장_종류_코드_명_count      8027067
대지_위치_count           8027067
도로명_대지_위치_count       7076491
건물_명_count            1160843
시군구_코드_count          8027067
법정동_코드_count          8027063
대지_구분_코드_count        8027059
번_count               8027067
지_count               8027055
특수지_명_count             33202
블록_count                12772
로트_count                 8734
외필지_수_count           8027067
새주소_도로_코드_count       7076491
새주소_법정동_코드_count      7074579
새주소_지상지하_코드_count     8024683
새주소_본_번_count         7174022
새주소_부_번_count         5375334
동_명_count             2199542
주_부속_구분_코드_count      8027067
주_부속_구분_코드_명_count    8026468
대지_면적(㎡)_count        8027067
건축_면적(㎡)_count        8027067
건폐_율(%)_count         8027067
연면적(㎡)_count          8027067
용적_률_산정_연면적(㎡)_count  8027067
용적_률(%)_count         8027067
구조_코드_count           8006871
구조_코드_명_co

(표제부에서 각 컬럼별 널이 아닌 값 수를 셈)
(표제부는 컬럼명에 특수문자(%)가 들어간 컬럼이 있어, 그대로 사용하면 에러가 발생함. 따옴표로 감싸줘야 함)

In [65]:
# 기본개요와 표제부 조인

# Drop the view if it already exists
con.execute("DROP VIEW IF EXISTS 기본개요_표제부")

# Create a view for the joined data
create_view_query = """
CREATE VIEW 기본개요_표제부 AS
SELECT 
    기본개요.*,
    표제부.\"동_명\",
    표제부.\"주_부속_구분_코드\",
    표제부.\"주_부속_구분_코드_명\",
    표제부.\"대지_면적(㎡)\",
    표제부.\"건축_면적(㎡)\",
    표제부.\"연면적(㎡)\",
    표제부.\"용적_률_산정_연면적(㎡)\",
    표제부.\"구조_코드\",
    표제부.\"주_용도_코드\",
    표제부.\"지붕_코드\",
    표제부.\"세대_수(세대)\",
    표제부.\"가구_수(가구)\",
    표제부.\"높이(m)\",
    표제부.\"지상_층_수\",
    표제부.\"지하_층_수\",
    표제부.\"부속_건축물_수\",
    표제부.\"부속_건축물_면적(㎡)\",
    표제부.\"총_동_연면적(㎡)\",
    표제부.\"허가_일\",
    표제부.\"착공_일\",
    표제부.\"사용승인_일\",
    표제부.\"허가번호_년\",
    표제부.\"호_수(호)\",
    표제부.\"생성_일자\"
FROM 기본개요
INNER JOIN 표제부
ON 기본개요.관리_건축물대장_PK = 표제부.관리_건축물대장_PK
"""
con.execute(create_view_query)

# Verify the view creation by querying it
view_result = con.execute("SELECT * FROM 기본개요_표제부 LIMIT 5").fetchdf()
print(view_result)

  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0  1124165449      112419517        1         일반        2      일반건축물   
1  1124165450           None        1         일반        2      일반건축물   
2  1124165453      112415487        1         일반        2      일반건축물   
3  1124165455      112415810        1         일반        2      일반건축물   
4  1124165456      112415810        1         일반        2      일반건축물   

                     대지_위치                        도로명_대지_위치  건물_명 시군구_코드  ...  \
0  경기도 화성시 양감면 사창리 660-3번지    경기도 화성시 안요골길82번길 43 (양감면 사창리)  None  41590  ...   
1    경기도 화성시 양감면 사창리 662번지  경기도 화성시 안요골길82번길 23-6 (양감면 사창리)  None  41590  ...   
2    경기도 화성시 양감면 사창리 665번지    경기도 화성시 안요골길82번길 22 (양감면 사창리)    다동  41590  ...   
3    경기도 화성시 양감면 사창리 669번지     경기도 화성시 안요골길82번길 1 (양감면 사창리)  None  41590  ...   
4    경기도 화성시 양감면 사창리 669번지     경기도 화성시 안요골길82번길 1 (양감면 사창리)  None  41590  ...   

  지하_층_수 부속_건축물_수 부속_건축물_면적(㎡) 총_동_연면적(㎡)      허가_일  착공_일    사용승인_일  허가번호_년  \
0

표제부에서 일부 컬럼만 사용

동_명
주_부속_구분_코드
주_부속_구분_코드_명
대지_면적(㎡)
건축_면적(㎡)
연면적(㎡)
용적_률_산정_연면적(㎡)
구조_코드
주_용도_코드
지붕_코드
세대_수(세대)
가구_수(가구)
높이(m)
지상_층_수
지하_층_수
부속_건축물_수
부속_건축물_면적(㎡)
총_동_연면적(㎡)
허가_일
착공_일
사용승인_일
허가번호_년
호_수(호)
생성_일자

In [66]:
row_count = con.execute("SELECT COUNT(*) FROM 기본개요_표제부").fetchone()[0]
print(f"Row count of 기본개요_표제부: {row_count}")

Row count of 기본개요_표제부: 8027067


표제부 테이블에 있는 모든 데이터가 기본개요와 잘 연계됨

In [ ]:
# 1) 테이블 스키마에서 컬럼 이름을 가져오고
cols_df = con.execute("PRAGMA table_info('기본개요_표제부')").fetchdf()
cols = cols_df["name"].tolist()

# 2) 각 컬럼에 대한 COUNT 문을 생성
# 표제부에는 컬럼명에 %가 들어간 경우가 있어, 오류가 발생함
# 이를 피하기 위해, 컬럼명을 따옴표로 감싸줌
count_exprs = [f'COUNT("{col}") AS "{col}"' for col in cols]

# 3) 최종 쿼리 조립
query = """
SELECT
    {exprs}
FROM 기본개요_표제부
""".format(exprs=",\n    ".join(count_exprs))

# 4) 실행 및 결과 확인
result = con.execute(query).fetchdf()

with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    # Display the result with all rows and columns
    print(result.T)

                      0
관리_건축물대장_PK     8027067
관리_상위_건축물대장_PK  2150212
대장_구분_코드        8027067
대장_구분_코드_명      8027067
대장_종류_코드        8027067
대장_종류_코드_명      8027067
대지_위치           8027067
도로명_대지_위치       7076491
건물_명            1160843
시군구_코드          8027067
법정동_코드          8027063
대지_구분_코드        8027059
번               8027067
지               8027055
특수지_명             33202
블록                12772
로트                 8734
외필지_수           8027067
새주소_도로_코드       7076491
새주소_법정동_코드      7074579
새주소_지상지하_코드     8024683
새주소_본_번         7174022
새주소_부_번         5375334
지역_코드           4018039
지구_코드            804783
구역_코드            862621
지역_코드_명         3994203
지구_코드_명          784397
구역_코드_명          849885
생성_일자           8027067
동_명             2199542
주_부속_구분_코드      8027067
주_부속_구분_코드_명    8026468
대지_면적(㎡)        8027067
건축_면적(㎡)        8027067
연면적(㎡)          8027067
용적_률_산정_연면적(㎡)  8027067
구조_코드           8006871
주_용도_코드         7997938
지붕_코드           7989535
세대_수(세대)        

기본개요와 잘 연계된 표제부에 기재된 컬럼별 데이터 기재 건수

총 8,027,067개 동 중 법정동 기재된 경우 8027063, 
구조_코드 8006871,
주_용도_코드 7997938,
지붕_코드 7989535,
사용승인일 7401141 등

데이터가 충분히 기재된 컬럼만 데이터 정제 과정에 활용



In [68]:
# Query to count rows where 사용승인일 is not 8 characters long
query = """
SELECT COUNT(*) AS invalid_사용승인일_count
FROM 기본개요_표제부
WHERE LENGTH(사용승인_일) != 8
"""

# Execute the query and fetch the result
invalid_count = con.execute(query).fetchone()[0]
print(f"Number of rows where 사용승인일 is not 8 characters long: {invalid_count}")

Number of rows where 사용승인일 is not 8 characters long: 28819


28819개 동은 사용승인일이 데이터 형식(8자리 숫자열)에 맞지 않게 기재되어 있음

In [71]:
# Query to count rows where 사용승인일 is larger than 20250229
query = """
SELECT COUNT(*) AS invalid_사용승인일_count
FROM 기본개요_표제부
WHERE 사용승인_일 > '20250229' and LENGTH(사용승인_일) = 8
"""

# Execute the query and fetch the result
invalid_count = con.execute(query).fetchone()[0]
print(f"Number of rows where 사용승인일 is larger than 20250229: {invalid_count}")

Number of rows where 사용승인일 is larger than 20250229: 54


54개 동은 사용승인일 데이터 기준 시점 이후로 기재되어 있음

In [ ]:
# Query to fetch rows where 사용승인_일 is greater than '20250229'
query = """
SELECT *
FROM 기본개요_표제부
WHERE 사용승인_일 > '20250229' and LENGTH(사용승인_일) = 8
"""

# Execute the query and fetch the result
result = con.execute(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(result)

        관리_건축물대장_PK   관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0    11921100195909   11921100195910        1         일반        2      일반건축물   
1    10801100229057   10801100229053        1         일반        2      일반건축물   
2        1205147112             None        1         일반        2      일반건축물   
3         123112518             None        1         일반        2      일반건축물   
4         111617947        111611754        1         일반        2      일반건축물   
5        1177123171             None        1         일반        2      일반건축물   
6        1177123713         11771452        1         일반        2      일반건축물   
7        1177123866         11771453        1         일반        2      일반건축물   
8        1177142115             None        1         일반        2      일반건축물   
9        1183132021             None        1         일반        2      일반건축물   
10      11861139567             None        1         일반        2      일반건축물   
11       1212115343             None    

전라남도 여수시 여천중학교의 예를 들면, 사용승인일이 '21120323'로 기재되어 있는데, 착공일이 '20111212'인 점을 고려할 때 2011년 12월 모일의 오기로 의심되나, 모든 데이터를 이렇게 정확하게 파악할 수는 없음.

In [76]:
# Query to count rows where 사용승인일 is less than 19000101
query = """
SELECT COUNT(*) AS invalid_사용승인일_count
FROM 기본개요_표제부
WHERE 사용승인_일 < '19000101' and LENGTH(사용승인_일) = 8
"""

# Execute the query and fetch the result
invalid_count = con.execute(query).fetchone()[0]
print(f"Number of rows where 사용승인일 is less than 19000101: {invalid_count}")

Number of rows where 사용승인일 is less than 19000101: 56477


56477개 동은 사용승인일 데이터가 1900년 이전 시점으로 기재되어 있음

In [ ]:
# Query to count rows where 사용승인일 is between 20241231 and 20250229
query = """
SELECT COUNT(*) AS invalid_사용승인일_count
FROM 기본개요_표제부
WHERE 사용승인_일 > '20241231' and 사용승인_일 <= '20250229' and LENGTH(사용승인_일) = 8
"""

# Execute the query and fetch the result
invalid_count = con.execute(query).fetchone()[0]
print(
    f"Number of rows where 사용승인일 is between 20241231 and 20250229: {invalid_count}"
)

Number of rows where 사용승인일 is between 20241231 and 20250229: 11585


11585개 동은 사용승인일이 2024년 말 이후로, 2024년 말 기준 분석에서는 제외가 필요함

위와 같은 여러 오류를 제외하여 사용승인일이 정상적으로 기재된 경우만 선별함

In [79]:
# Query to count rows where 사용승인일 is between 19000101 and 20241231
query = """
SELECT COUNT(*) AS valid_사용승인일_count
FROM 기본개요_표제부
WHERE 사용승인_일 > '19000101' and 사용승인_일 <= '20241231' and LENGTH(사용승인_일) = 8
"""

# Execute the query and fetch the result
valid_count = con.execute(query).fetchone()[0]
print(
    f"Number of rows where 사용승인일 is between 19000101 and 20241231: {valid_count}"
)

Number of rows where 사용승인일 is between 19000101 and 20241231: 7303104


1900년 이후 2024년 말까지 사용승인된 건축물 수는 7303104 동

1차로 이 건축물만 사용

In [81]:
# Drop the view if it already exists
con.execute("DROP VIEW IF EXISTS 기본개요_표제부_사용승인일_정상")

# Create a view with the specified conditions
create_view_query = """
CREATE VIEW 기본개요_표제부_사용승인일_정상 AS
SELECT *
FROM 기본개요_표제부
WHERE 사용승인_일 > '19000101' 
    AND 사용승인_일 <= '20241231' 
    AND LENGTH(사용승인_일) = 8
"""
con.execute(create_view_query)

# Verify the view creation by querying it
view_result = con.execute(
    "SELECT * FROM 기본개요_표제부_사용승인일_정상 LIMIT 5"
).fetchdf()

with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(view_result)

  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0  1124165449      112419517        1         일반        2      일반건축물   
1  1124165450           None        1         일반        2      일반건축물   
2  1124165453      112415487        1         일반        2      일반건축물   
3  1124165455      112415810        1         일반        2      일반건축물   
4  1124165456      112415810        1         일반        2      일반건축물   

                     대지_위치                        도로명_대지_위치  건물_명 시군구_코드  \
0  경기도 화성시 양감면 사창리 660-3번지    경기도 화성시 안요골길82번길 43 (양감면 사창리)  None  41590   
1    경기도 화성시 양감면 사창리 662번지  경기도 화성시 안요골길82번길 23-6 (양감면 사창리)  None  41590   
2    경기도 화성시 양감면 사창리 665번지    경기도 화성시 안요골길82번길 22 (양감면 사창리)    다동  41590   
3    경기도 화성시 양감면 사창리 669번지     경기도 화성시 안요골길82번길 1 (양감면 사창리)  None  41590   
4    경기도 화성시 양감면 사창리 669번지     경기도 화성시 안요골길82번길 1 (양감면 사창리)  None  41590   

  법정동_코드 대지_구분_코드     번     지 특수지_명    블록    로트  외필지_수     새주소_도로_코드  \
0  40022        0  0660  0003  None  N

In [83]:
# Query to fetch unique values for 구조_코드
query = """
SELECT 구조_코드, COUNT(*) AS count
FROM 기본개요_표제부_사용승인일_정상
GROUP BY 구조_코드
ORDER BY count DESC
"""

# Execute the query and fetch the result
unique_values_구조_코드 = con.execute(query).fetchdf()

# Display the result
print(unique_values_구조_코드)

   구조_코드    count
0     21  2014923
1     11  1502117
2     51  1017669
3     32   921047
4     31   762379
5     12   590672
6     33   179454
7     19   169838
8     39    25750
9     10    19657
10    42    18358
11    13    12166
12    41    11027
13    74    10750
14    99    10507
15    63     5301
16    50     4621
17    61     4410
18    52     4356
19    22     4075
20  None     4022
21    29     2122
22    30     1881
23    20     1312
24    23      841
25    34      823
26    53      805
27    37      669
28    43      287
29    72      257
30    49      221
31    81      149
32    26      136
33    14       94
34    35       82
35    40       75
36    24       75
37    36       65
38    90       56
39    17       22
40    27       12
41    60        9
42    80        5
43              3
44    70        2
45     `        1
46    01        1


구조코드: 2자리 숫자열

구조코드 없는 경우 4022, 빈 문자열인 경우 3, 특수문자 1건

구조코드가 없으면 뺄 것인지?

In [86]:
# Query to fetch unique values for 주_용도_코드
query = """
SELECT 주_용도_코드, COUNT(*) AS count
FROM 기본개요_표제부_사용승인일_정상
GROUP BY 주_용도_코드
ORDER BY count DESC
"""

# Execute the query and fetch the result
unique_values_주_용도_코드 = con.execute(query).fetchdf()
# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    # Display the result with all rows and columns
    print(unique_values_주_용도_코드)

    주_용도_코드    count
0     01000  3831218
1     04000   691059
2     02000   570702
3     03000   556012
4     21000   417057
5     17000   389577
6     18000   378094
7     10000    74222
8     11000    46348
9     15000    46117
10    06000    36381
11    19000    36300
12    14000    36032
13    20000    35986
14    23000    20826
15    05000    14365
16    13000    13571
17    30000    12867
18    22000    12125
19    07000    11813
20    Z3000     9595
21    27000     8878
22    09000     7207
23    31000     6995
24     None     5171
25    16000     5066
26    33000     4835
27    12000     4803
28    25000     4441
29    08000     3681
30    24000     3197
31    Z8000     3181
32    Z5000     2018
33    26000     1579
34    29000      513
35    Z6000      395
36    Z9000      222
37       JY       92
38    21101       67
39    06107       47
40    Z7001       43
41    02003       38
42    03001       29
43    01001       26
44    04010       26
45    32000       26
46    18999  

주용도코드: 5자리 숫자열, 뒷 3자리 000: 대분류 의미

주용도코드 없는 경우 5171

구 4자리 용도 코드 (Z로 시작) 별도 처리 필요

5자리 숫자열에 맞지 않는 경우 존재(JY)

어차피 층별개요의 주용도코드 사용 예정

In [87]:
# Query to fetch unique values for 지붕_코드
query = """
SELECT 지붕_코드, COUNT(*) AS count
FROM 기본개요_표제부_사용승인일_정상
GROUP BY 지붕_코드
ORDER BY count DESC
"""

# Execute the query and fetch the result
unique_values_지붕_코드 = con.execute(query).fetchdf()
# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    # Display the result with all rows and columns
    print(unique_values_지붕_코드)

   지붕_코드    count
0     10  3291296
1     90  2387188
2     30   835015
3     20   773758
4   None    15733
5     15       80
6     13       18
7     01        4
8     21        4
9     99        2
10    06        2
11     1        2
12     0        1
13    32        1


지붕 코드: 2자리 숫자열

지붕코드 없는 경우 15733

구조코드가 없으면 뺄 것인지?

층별개요 검증 시작

In [130]:
# Drop the view if it already exists
con.execute("DROP VIEW IF EXISTS 층별개요_필요컬럼")

# Create a view with the specified columns
create_view_query = """
CREATE VIEW 층별개요_필요컬럼 AS
SELECT 
    관리_건축물대장_PK,
    대지_위치,
    건물_명,
    층_구분_코드,
    층_구분_코드_명,
    층_번호,
    층_번호_명,
    구조_코드,
    주_용도_코드,
    주_용도_코드_명,
    "면적(㎡)",
    주_부속_구분_코드,
    면적_제외_여부
FROM 층별개요
"""
con.execute(create_view_query)

# Verify the view creation by querying it
view_result = con.execute("SELECT * FROM 층별개요_필요컬럼 LIMIT 5").fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(view_result)

  관리_건축물대장_PK                   대지_위치    건물_명 층_구분_코드 층_구분_코드_명  층_번호 층_번호_명  \
0    10021950     서울특별시 종로구 숭인동 201번지    None      20        지상     1     1층   
1    10021974    서울특별시 종로구 숭인동 1132번지    None      20        지상     1     1층   
2   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      10        지하     1   지하1층   
3   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      20        지상     1     1층   
4   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      20        지상     2     2층   

  구조_코드 주_용도_코드    주_용도_코드_명   면적(㎡) 주_부속_구분_코드 면적_제외_여부  
0    51   01001         단독주택   29.75          0     None  
1    12   04999  기타제2종근린생활시설   36.26          0     None  
2    21   02003        다세대주택  161.89          0     None  
3    21   02003        다세대주택  137.95          0     None  
4    21   02003        다세대주택  137.95          0     None  


In [131]:
# Query to count values of 면적_제외_여부
query = """
SELECT 면적_제외_여부, COUNT(*) AS count
FROM 층별개요_필요컬럼
GROUP BY 면적_제외_여부
ORDER BY count DESC
"""

# Execute the query and fetch the result
result = con.execute(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(result)

  면적_제외_여부     count
0     None  12816630
1        0   7954285
2        1    277800


면적 제외 여부는 기재되지 않은 경우가 12816630,
미제외(0) 7954285,
제외(1) 277800

기재되지 않은 경우 미제외로 판단하여 진행, 제외로 기재된 경우만 제외함

In [132]:
# Query to count values of 주_용도_코드
query = """
SELECT 주_용도_코드, COUNT(*) AS count
FROM 층별개요_필요컬럼
GROUP BY 주_용도_코드
ORDER BY count DESC
"""

# Execute the query and fetch the result
result = con.execute(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(result)

    주_용도_코드    count
0     01001  6118573
1     02001  2877653
2     01003  1923975
3     02003  1261033
4     03001   903494
5     18001   655616
6     04402   600119
7     04001   563381
8     21101   493776
9     17100   432244
10    03999   330306
11    17999   284297
12    02005   234621
13    04999   221739
14    14202   213301
15    04005   196149
16    21999   164832
17    02002   157461
18    18999   135702
19    03002   116898
20    03005   112282
21    04010   108025
22    04499    90362
23    18101    90040
24    15102    87025
25    14204    82929
26    01002    80811
27    17301    77765
28    10101    67484
29    11201    65775
30    14299    63723
31    20001    59401
32    06101    58931
33    03021    54167
34    04006    49329
35    03007    43710
36    03011    41623
37    03199    40845
38    10103    38953
39    10999    36370
40    10104    35347
41    04002    31598
42    02006    30457
43    10003    30258
44    19001    29711
45    10102    29482
46    23003  

주용도코드: 5자리 숫자열, 소분류

주용도코드 없는 경우 25487

4자리 코드 (b, k, Z로 시작) 별도 처리 필요

5자리 숫자열에 맞지 않는 경우 존재(창, 홀, 점)

In [133]:
# Query to count rows where 면적(㎡) is less than 0
query = """
SELECT COUNT(*) AS negative_area_count
FROM 층별개요_필요컬럼
WHERE "면적(㎡)" < 0
"""

# Execute the query and fetch the result
negative_area_count = con.execute(query).fetchone()[0]
print(f"Number of rows where 면적(㎡) is less than 0: {negative_area_count}")

Number of rows where 면적(㎡) is less than 0: 195


면적이 0보다 작은 경우 존재. 제외 필요

In [134]:
# Query to fetch the "면적(㎡)" column
query = """
SELECT "면적(㎡)"
FROM 층별개요_필요컬럼
WHERE "면적(㎡)" IS NOT NULL
"""

# Fetch the data
area_data = con.execute(query).fetchdf()

# Calculate statistics
max_value = area_data["면적(㎡)"].max()
mean_value = area_data["면적(㎡)"].mean()
percentile_1m = area_data["면적(㎡)"].quantile(0.999999)  # 1 - 1/1,000,000
percentile_100k = area_data["면적(㎡)"].quantile(0.99999)
percentile_10k = area_data["면적(㎡)"].quantile(0.9999)
percentile_1k = area_data["면적(㎡)"].quantile(0.999)  # 1 - 1/1,000
percentile_1h = area_data["면적(㎡)"].quantile(0.99)  # 1 - 1/100

# Display the results
print(f"Max value:           {max_value: 15.2f}")
print(f"Mean value:          {mean_value: 15.2f}")
print(f"0.999999 percentile: {percentile_1m: 15.2f}")
print(f"0.99999 percentile:  {percentile_100k: 15.2f}")
print(f"0.9999 percentile:   {percentile_10k: 15.2f}")
print(f"0.999 percentile:    {percentile_1k: 15.2f}")
print(f"0.99 percentile:     {percentile_1h: 15.2f}")


Max value:              188957549.00
Mean value:                   227.62
0.999999 percentile:      1824460.16
0.99999 percentile:         71130.88
0.9999 percentile:          29026.12
0.999 percentile:            8396.60
0.99 percentile:             1733.89


In [135]:
# Query to fetch data sorted by 면적(㎡) in descending order
query = """
SELECT *
FROM 층별개요_필요컬럼
ORDER BY "면적(㎡)" DESC
LIMIT 100
"""

# Execute the query and fetch the result
sorted_data = con.execute(query).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
    "display.float_format",
    "{:,.2f}".format,  # Format floats as non-scientific with 2 decimal places
):
    print(sorted_data)

        관리_건축물대장_PK                           대지_위치                건물_명  \
0    11231100281894            경기도 김포시 운양동 1301-1번지          한강신도시 롯데캐슬   
1         112512587               경기도 광주시 태전동 687번지          쌍용 스윗닷홈아파트   
2         118014186          충청남도 서천군 장항읍 신창리 399번지                한솔제지   
3   108215000112847               세종특별자치시 새롬동 554번지            새뜸마을 4단지   
4    10591100243012             인천광역시 부평구 십정동 607번지            동암신동아아파트   
5    10461100239805             대구광역시 동구 신서동 1188번지       대구혁신도시 비젼스퀘어2   
6         105917500          인천광역시 부평구 삼산동 454-13번지                삼산타운   
7    10331100178301           부산광역시 동래구 명륜동 756-1번지     동래 센트럴파크 하이츠 2차   
8    10841100363865            경기도 수원시 영통구 하동 989번지             원희캐슬 광교   
9        1124142100           경기도 화성시 비봉면 양노리 850번지             천산 스카이빌   
10        106917066           광주광역시 광산구 신촌동 826-1번지            신촌동대주아파트   
11        106917065           광주광역시 광산구 신촌동 826-1번지            신촌동대주아파트   
12       1042113207      

지하 3층 면적이 188,957,549 으로 기재된 경기 김포 한강신도시 롯데캐슬의 경우, 대지 면적이 65,940 에 불과하여 층별개요에 기재된 면적은 현실적으로 불가능한 수치

이러한 오류 사례를 거르기 위하여 개별 사례를 검토 후 층별 면적이 500,000 이상인 경우 (0.001% 미만 사례) 제외하도록 하였다.


In [136]:
# Drop the view if it already exists
con.execute("DROP VIEW IF EXISTS 층별개요_필터링")

# Create a view with the specified conditions
create_view_query = """
CREATE VIEW 층별개요_필터링 AS
SELECT *
FROM 층별개요_필요컬럼
WHERE (면적_제외_여부 IS NULL OR 면적_제외_여부 != 1)
  AND "면적(㎡)" > 0
  AND "면적(㎡)" <= 500000
"""
con.execute(create_view_query)

# Verify the view creation by querying it
view_result = con.execute("SELECT * FROM 층별개요_필터링 LIMIT 5").fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(view_result)

  관리_건축물대장_PK                   대지_위치    건물_명 층_구분_코드 층_구분_코드_명  층_번호 층_번호_명  \
0    10021950     서울특별시 종로구 숭인동 201번지    None      20        지상     1     1층   
1    10021974    서울특별시 종로구 숭인동 1132번지    None      20        지상     1     1층   
2   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      10        지하     1   지하1층   
3   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      20        지상     1     1층   
4   100211003  서울특별시 종로구 부암동 129-10번지  부암스카이빌      20        지상     2     2층   

  구조_코드 주_용도_코드    주_용도_코드_명   면적(㎡) 주_부속_구분_코드 면적_제외_여부  
0    51   01001         단독주택   29.75          0     None  
1    12   04999  기타제2종근린생활시설   36.26          0     None  
2    21   02003        다세대주택  161.89          0     None  
3    21   02003        다세대주택  137.95          0     None  
4    21   02003        다세대주택  137.95          0     None  


In [138]:
# Drop the view if it already exists
con.execute("DROP VIEW IF EXISTS 기본개요_표제부_층별개요_조인")

# Create a view for the joined data
create_view_query = """
CREATE VIEW 기본개요_표제부_층별개요_조인 AS
SELECT 
    a.*,
    b.층_구분_코드,
    b.층_구분_코드_명,
    b.층_번호,
    b.층_번호_명,
    b.구조_코드 AS 층_구조_코드,
    b.주_용도_코드 AS 층_주_용도_코드,
    b.주_용도_코드_명 AS 층_주_용도_코드_명,
    b."면적(㎡)" AS 층_면적,
    b.주_부속_구분_코드 AS 층_주_부속_구분_코드,
    b.면적_제외_여부 AS 층_면적_제외_여부
FROM 기본개요_표제부_사용승인일_정상 a
INNER JOIN 층별개요_필터링 b
ON a.관리_건축물대장_PK = b.관리_건축물대장_PK
"""
con.execute(create_view_query)

# Verify the view creation by querying it
view_result = con.execute(
    "SELECT * FROM 기본개요_표제부_층별개요_조인 LIMIT 5"
).fetchdf()

# Display the result
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
):
    print(view_result)

  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0  1110126357           None        2         집합        3        표제부   
1  1110126357           None        2         집합        3        표제부   
2  1110126374           None        2         집합        3        표제부   
3  1110126374           None        2         집합        3        표제부   
4  1110126374           None        2         집합        3        표제부   

                      대지_위치                          도로명_대지_위치    건물_명 시군구_코드  \
0  경기도 남양주시 화도읍 녹촌리 245-7번지  경기도 남양주시 경춘로1842번길 31-1 (화도읍 녹촌리)  고광빌라다동  41360   
1  경기도 남양주시 화도읍 녹촌리 245-7번지  경기도 남양주시 경춘로1842번길 31-1 (화도읍 녹촌리)  고광빌라다동  41360   
2    경기도 남양주시 다산동 4265-32번지        경기도 남양주시 미금로19번길 12-2 (다산동)    None  41360   
3    경기도 남양주시 다산동 4265-32번지        경기도 남양주시 미금로19번길 12-2 (다산동)    None  41360   
4    경기도 남양주시 다산동 4265-32번지        경기도 남양주시 미금로19번길 12-2 (다산동)    None  41360   

  법정동_코드 대지_구분_코드     번     지 특수지_명    블록    로트  외필지_수     새주소_도로_코드  \
0  25629

In [139]:
# Query to count rows in 기본개요_표제부_층별개요_조인
query = """
SELECT COUNT(*) AS row_count
FROM 기본개요_표제부_층별개요_조인
"""

# Execute the query and fetch the result
row_count = con.execute(query).fetchone()[0]
print(f"Row count of 기본개요_표제부_층별개요_조인: {row_count}")

Row count of 기본개요_표제부_층별개요_조인: 19614969


In [140]:
# Query to calculate the sum of 층_면적
query = """
SELECT SUM("층_면적") AS total_floor_area
FROM 기본개요_표제부_층별개요_조인
"""

# Execute the query and fetch the result
total_floor_area = con.execute(query).fetchone()[0]
print(f"Total floor area: {total_floor_area:,.0f}")

Total floor area: 4,372,971,283


분석 대상 건축물 층별 면적 합계: 4,372,971,283

2024년 건축물 현황 시도별 건축물 연면적 합계: 4,314,987,939 

일부 오차가 있으며, 통계 집계 기준이나 오류 정정 규칙 등 차이로 인한 것일 수 있음

건축물 현황은 부속 건축물을 집계하지 않으나, 본 연구에서는 재조달원가의 정확한 산정을 위하여 부속 건축물 면적도 포함하였음

In [143]:
# Query to calculate the total area by 층_주_용도_코드 and 층_주_용도_코드명
query = """
SELECT 
    층_주_용도_코드,
    층_주_용도_코드_명,
    SUM("층_면적") AS total_area
FROM 기본개요_표제부_층별개요_조인
GROUP BY 층_주_용도_코드, 층_주_용도_코드_명
ORDER BY total_area DESC
"""

# Execute the query and fetch the result
area_summary = con.execute(query).fetchdf()

# Save the result to a CSV file
output_file = data_root / "floor_usage_area_summary.csv"
area_summary.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Summary saved to {output_file}")

Summary saved to D:\데이터\건축데이터 건축허브 개방데이터\floor_usage_area_summary.csv


In [151]:
# Query to calculate the total area by 시군구_코드
query = """
SELECT 
    시군구_코드,
    SUM("층_면적") AS total_area
FROM 기본개요_표제부_층별개요_조인
GROUP BY 시군구_코드
ORDER BY total_area DESC
"""

# Execute the query and fetch the result
area_summary = con.execute(query).fetchdf()

# Load the 시군구 코드 to 시군구명 mapping from the CSV file
sgg_mapping = pd.read_csv(
    "../data/processed/code_sgg.csv", dtype=str, encoding="utf-8-sig"
)

# Merge the area summary with the 시군구명 mapping
area_summary = area_summary.merge(
    sgg_mapping, how="left", left_on="시군구_코드", right_on="시군구코드"
)

# Drop the redundant 시군구코드 column after the merge
area_summary = area_summary.drop(columns=["시군구코드"])
# Save the result to a CSV file
output_file = data_root / "floor_area_by_region_summary.csv"
area_summary.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Summary saved to {output_file}")

Summary saved to D:\데이터\건축데이터 건축허브 개방데이터\floor_area_by_region_summary.csv


In [152]:
# Query to calculate the total area by 구조_코드
query = """
SELECT 
    구조_코드,
    SUM("층_면적") AS total_area
FROM 기본개요_표제부_층별개요_조인
GROUP BY 구조_코드
ORDER BY total_area DESC
"""

# Execute the query and fetch the result
area_by_structure_code = con.execute(query).fetchdf()

# Save the result to a CSV file
output_file = data_root / "area_by_structure_code.csv"
area_by_structure_code.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Result saved to {output_file}")


Result saved to D:\데이터\건축데이터 건축허브 개방데이터\area_by_structure_code.csv


In [153]:
# Query to calculate the total area by 지붕_코드
query = """
SELECT 
    지붕_코드,
    SUM("층_면적") AS total_area
FROM 기본개요_표제부_층별개요_조인
GROUP BY 지붕_코드
ORDER BY total_area DESC
"""

# Execute the query and fetch the result
area_by_roof_code = con.execute(query).fetchdf()

# Save the result to a CSV file
output_file = data_root / "area_by_roof_code.csv"
area_by_roof_code.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Result saved to {output_file}")

Result saved to D:\데이터\건축데이터 건축허브 개방데이터\area_by_roof_code.csv


In [154]:
# Define the output file path for the Parquet file
output_parquet_file = data_root / "기본개요_표제부_층별개요_조인.parquet"

# Query to fetch all data from the view
query = """
SELECT *
FROM 기본개요_표제부_층별개요_조인
"""

# Execute the query and fetch the result as a DataFrame
data = con.execute(query).fetchdf()

# Save the DataFrame to a Parquet file
data.to_parquet(output_parquet_file, index=False)

print(f"Data saved to {output_parquet_file}")

Data saved to D:\데이터\건축데이터 건축허브 개방데이터\기본개요_표제부_층별개요_조인.parquet
